In [7]:
!python main.py --train --sine-data

^C


In [9]:
!python old_main.py

In [1]:
import numpy as np

In [6]:
n_sequences = 5
max_degree = 5

degrees = np.random.choice([0, 1], size=(n_sequences, max_degree))

In [7]:
degrees

array([[1, 1, 1, 1, 0],
       [1, 0, 1, 0, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1]])